# Representation Learning with Autoencoders & GANs
## Chapter 17 - Generative Models Implementation Guide

## 1. Introduction to Representation Learning

**Key Concepts**:
- Unsupervised learning of efficient data encodings
- Dimensionality reduction with neural networks
- Two main approaches:
  - Autoencoders (deterministic)
  - Generative Adversarial Networks (probabilistic)

**Applications**:
- Data compression
- Anomaly detection
- Image generation
- Feature learning

## 2. Autoencoders

### 2.1 Basic Architecture
\[
\text{Input } x \rightarrow \text{Encoder } E(x) \rightarrow \text{Latent } z \rightarrow \text{Decoder } D(z) \rightarrow \text{Reconstruction } \hat{x}
\]

**Components**:
- Encoder: $z = E(x)$
- Decoder: $\hat{x} = D(z)$
- Loss: $\mathcal{L}(x, \hat{x}) = ||x - D(E(x))||^2$

In [ ]:
from tensorflow.keras import layers, Model

# Simple Autoencoder implementation
input_dim = 784  # MNIST images
latent_dim = 32

# Encoder
encoder_input = layers.Input(shape=(input_dim,))
x = layers.Dense(256, activation='relu')(encoder_input)
z = layers.Dense(latent_dim, activation='relu')(x)
encoder = Model(encoder_input, z, name='encoder')

# Decoder
decoder_input = layers.Input(shape=(latent_dim,))
x = layers.Dense(256, activation='relu')(decoder_input)
output = layers.Dense(input_dim, activation='sigmoid')(x)
decoder = Model(decoder_input, output, name='decoder')

# Autoencoder
autoencoder_input = layers.Input(shape=(input_dim,))
encoded = encoder(autoencoder_input)
decoded = decoder(encoded)
autoencoder = Model(autoencoder_input, decoded, name='autoencoder')

autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.summary()

### 2.2 Variational Autoencoders (VAEs)

**Key Differences**:
- Encoder outputs parameters of probability distribution ($\mu$, $\sigma$)
- Latent space sampling: $z = \mu + \sigma \odot \epsilon$
- Loss function:
\[
\mathcal{L} = \mathbb{E}[\log p(x|z)] - D_{KL}(q(z|x) || p(z))
\]

In [ ]:
class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# VAE Encoder
encoder_input = layers.Input(shape=(input_dim,))
x = layers.Dense(256, activation='relu')(encoder_input)
z_mean = layers.Dense(latent_dim)(x)
z_log_var = layers.Dense(latent_dim)(x)
z = Sampling()([z_mean, z_log_var])
encoder = Model(encoder_input, [z_mean, z_log_var, z], name='encoder')

# VAE Decoder (same as before)

# VAE Model
outputs = decoder(encoder(encoder_input)[2])
vae = Model(encoder_input, outputs, name='vae')

# Add KL divergence loss
kl_loss = -0.5 * tf.reduce_mean(z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
vae.add_loss(kl_loss)
vae.compile(optimizer='adam')
vae.summary()

## 3. Generative Adversarial Networks (GANs)

### 3.1 GAN Architecture
\[
\min_G \max_D V(D,G) = \mathbb{E}_{x\sim p_{data}}[\log D(x)] + \mathbb{E}_{z\sim p_z}[\log(1-D(G(z)))]
\]

**Components**:
- Generator: $G(z) \rightarrow \hat{x}$
- Discriminator: $D(x) \rightarrow [0,1]$
- Adversarial training process

In [ ]:
# GAN Implementation
latent_dim = 100

# Generator
generator = tf.keras.Sequential([
    layers.Dense(256, activation='relu', input_dim=latent_dim),
    layers.BatchNormalization(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(784, activation='tanh'),  # MNIST 28x28
    layers.Reshape((28, 28, 1))
], name='generator')

# Discriminator
discriminator = tf.keras.Sequential([
    layers.Flatten(input_shape=(28, 28, 1)),
    layers.Dense(512, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(1, activation='sigmoid')
], name='discriminator')

# Combined GAN
gan = tf.keras.Sequential([generator, discriminator])

# Compile models
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
discriminator.trainable = False
gan.compile(optimizer='adam', loss='binary_crossentropy')

generator.summary()

### 3.2 GAN Training Loop

Key steps:
1. Train discriminator on real images (label 1)
2. Train discriminator on fake images (label 0)
3. Train generator to fool discriminator

In [ ]:
# Simplified training loop
def train_gan(generator, discriminator, gan, dataset, epochs=10, batch_size=128):
    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    for epoch in range(epochs):
        for batch in dataset:
            # Train Discriminator
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            gen_images = generator.predict(noise)
            
            # Train on real and fake images
            d_loss_real = discriminator.train_on_batch(batch, valid)
            d_loss_fake = discriminator.train_on_batch(gen_images, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            
            # Train Generator
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            g_loss = gan.train_on_batch(noise, valid)
            
        print(f"Epoch {epoch+1}, D Loss: {d_loss[0]}, G Loss: {g_loss}")

# Note: In practice, you'd use proper batching and normalization

## 4. Advanced Generative Models

### 4.1 Conditional GANs
- Generate samples conditioned on class labels
- Useful for controlled generation

### 4.2 StyleGAN
- Progressive growing for high-res images
- Style-based generator architecture
- Fine-grained control over image features

## 5. Applications

### 5.1 Data Augmentation
- Generate synthetic training samples
- Balance imbalanced datasets

### 5.2 Image-to-Image Translation
- CycleGAN for unpaired translation
- Pix2Pix for paired translation

## 6. Exercises

1. Train an autoencoder on MNIST and visualize reconstructions
2. Implement a VAE and sample from the latent space
3. Train a DCGAN to generate faces (use CelebA dataset)
4. Experiment with different GAN architectures (WGAN, CGAN)
5. Use t-SNE to visualize autoencoder latent spaces

## 7. Key Takeaways

- Autoencoders learn compressed representations through reconstruction
- VAEs enable probabilistic sampling from latent space
- GANs generate realistic samples through adversarial training
- Modern architectures (StyleGAN, VQ-VAE) push quality boundaries
- Generative models have wide applications from art to data augmentation